In [ ]:
import sys
import os

sys.path.append(os.path.abspath('..'))
from scripts.build_index import build_index
from src.utils.call_api.schema import EmbeddingEngineEnum
from src.retrievers.models.vector_retriever import VectorRetriever
from configulator.config import default_config

In [ ]:
cfg = default_config()

In [ ]:
augmented_index_df = build_index(mode=cfg.mode, chunksize=cfg.chunksize, overlap=cfg.overlap)

# normal index
normal_index = augmented_index_df.drop_duplicates(subset='text')
vecretriever = VectorRetriever(normal_index, model=EmbeddingEngineEnum.Large)

# augmented index検索
aug_vecretriever = VectorRetriever(augmented_index_df,
                                   emb_column_name='augmented_embedding',
                                   target_column_name='augmented_text',
                                   model=EmbeddingEngineEnum.Large
                                   )
# save index
vec_index_df = aug_vecretriever.index_df.merge(vecretriever.index_df[['novel_id', 'chunk_id', 'embedding']], on=['novel_id', 'chunk_id'])
index_path = cfg.index_path
vec_index_df.to_csv(index_path)